# Assingment 2: Explainable AI and Hyperparameter Optimization

Samih Amer

In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 72
import numpy as np
import pandas as pd, re
import re
from collections import defaultdict
from numpy.linalg import norm
from scipy.spatial.distance import cosine 
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import gzip, io
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import adjusted_rand_score

In [6]:
df_movies = pd.read_csv(r'/Users/samihamer/Desktop/Advanced Applied AI Summer 2025/Module 2/movielens/movielens_movie.csv')
df_ratings = pd.read_csv(r'/Users/samihamer/Desktop/Advanced Applied AI Summer 2025/Module 2/movielens/movielens_rating.csv')

tags = pd.read_csv(r'/Users/samihamer/Desktop/Advanced Applied AI Summer 2025/Module 2/movielens/movielens_genome_tags.csv')
scores =  pd.read_csv(r'/Users/samihamer/Desktop/Advanced Applied AI Summer 2025/Module 2/movielens/movielens_genome_scores.csv')

genres_df = df_movies[['movieId', 'genres']].copy()

## Problem 1 - Rework the movielens dataset to determine a set of 19 tags that explains one of the 19 Genres while having an inert relation with the rest of the genres

In [7]:
GENRES = ['Action','Adventure','Animation',"Children",'Comedy','Crime',
          'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical',
          'Mystery','Romance','Sci-Fi','Thriller','War','Western','IMAX']

genres_df['genres'] = genres_df['genres'].fillna('')          
genres_df = genres_df[genres_df['genres'].str.contains('|'.join(GENRES))]

Explode genre list -> one row per (movie, genre)

In [8]:
gexpl = (genres_df.assign(genres=genres_df['genres'].str.split('|'))
                    .explode('genres')
                    .query("genres in @GENRES"))

Merge relevance scores with exploded genres

In [9]:
merge = gexpl[['movieId','genres']].merge(scores, on='movieId', how='inner')
merge = merge.merge(tags, on='tagId', how='left')

Sum relevance per (genre, tag)

In [10]:
grp = merge.groupby(['genres','tag']).agg(rel_sum=('relevance','sum'))

Probability P(tag|genre) ~ relevance_sum / total_relevance_in_genre

In [11]:
tot = grp.groupby(level=0)['rel_sum'].transform('sum')
p_tag_genre = grp['rel_sum'] / tot

In [12]:
p_tag_all   = (merge.groupby('tag')['relevance'].sum() /
               merge['relevance'].sum())
p_tag_other = lambda g: ( (merge[merge['genres']!=g]
                           .groupby('tag')['relevance']
                           .sum()) /
                         merge[merge['genres']!=g]['relevance'].sum() )

Pick best tag per genre

In [15]:
best_tags = {}
norm = lambda s: re.sub(r'[^a-z]', '', s.lower())   # letters only

for g in GENRES:
    s = p_tag_genre.xs(g) - p_tag_other(g)          # score
    g_norm = norm(g)

    for tag in s.sort_values(ascending=False).index:
        if g_norm not in norm(tag):                 # ← stricter check
            best_tags[g] = tag
            break

In [16]:
best = pd.Series(best_tags).rename_axis('Genre').to_frame('Representative tag')
display(best)

,Representative tag
Genre,
Action,fight scenes
Adventure,action
Animation,animated
Children,kids
Comedy,funny
Crime,murder
Documentary,politics
Drama,intimate
Fantasy,magic
